In [ ]:
# code to render stream

In [ ]:
!pip install aiortc aiohttp --quiet

In [ ]:
import aiortc, aiohttp
import asyncio, cv2, numpy as np, aiohttp, logging, base64
from aiortc import RTCPeerConnection, RTCSessionDescription
from aiortc.mediastreams import MediaStreamError
from IPython.display import display, HTML


WHEP_URL = "https://url/whep"
DISPLAY_FPS = 15


logging.getLogger("aiortc.codecs.h264").setLevel(logging.ERROR)
# logging.basicConfig(level=logging.WARNING)
# logging.getLogger("aiortc").setLevel(logging.WARNING)

class WebRTCStreamViewer:
    def __init__(self, whep_url):
        self.whep_url = whep_url
        self.pc = RTCPeerConnection()
        self.done = asyncio.Event()
        self.lock = asyncio.Lock()
        self.latest_frame = None


        @self.pc.on("track")
        async def on_track(track):
            if track.kind == "video":
                asyncio.create_task(self._frame_receiver_task(track))

        @self.pc.on("connectionstatechange")
        async def on_connectionstatechange():
            print(f"Connection state is {self.pc.connectionState}")
            if self.pc.connectionState in ["failed", "closed", "disconnected"]:
                self.done.set()


    async def _frame_receiver_task(self, track):
        while not self.done.is_set():
            try:
                frame = await track.recv()
                img = frame.to_ndarray(format="bgr24")
                async with self.lock:
                    self.latest_frame = img
            except MediaStreamError:
                return


    async def _display_loop_task(self):
        display_handle = display(HTML('<img>'), display_id=True)

        while not self.done.is_set():
            frame_to_display = None
            async with self.lock:
                if self.latest_frame is not None:

                    frame_to_display = self.latest_frame.copy()

            if frame_to_display is not None:
                # --- DEPTH ANALYSIS CODE GOES HERE ---
                processed_frame = frame_to_display
                # ---


                _, buffer = cv2.imencode('.jpg', processed_frame)
                b64_str = base64.b64encode(buffer).decode('utf-8')
                data_url = f"data:image/jpeg;base64,{b64_str}"
                display_handle.update(HTML(f'<img src="{data_url}" style="width: 80%;" />'))


            await asyncio.sleep(1 / DISPLAY_FPS)

    async def run(self):
        self.pc.addTransceiver("video", direction="recvonly")
        offer = await self.pc.createOffer()
        await self.pc.setLocalDescription(offer)

        try:
            display_task = asyncio.create_task(self._display_loop_task())
            async with aiohttp.ClientSession() as session:
                print(f"Connecting to {self.whep_url}...")
                async with session.post(self.whep_url, data=self.pc.localDescription.sdp,
                                        headers={"Content-Type": "application/sdp"}, timeout=15) as resp:
                    if resp.status == 201:
                        # print("WHEP connection accepted.")
                        answer_sdp = await resp.text()
                        await self.pc.setRemoteDescription(RTCSessionDescription(sdp=answer_sdp, type="answer"))
                    else:
                        # print(f"Server error: {resp.status} {await resp.text()}")
                        self.done.set()


            await self.done.wait()

        finally:
            # print("Shutting down")
            if 'display_task' in locals() and not display_task.done():
                display_task.cancel()
            if self.pc.connectionState != "closed":
                await self.pc.close()

async def main():
    viewer = WebRTCStreamViewer(WHEP_URL)
    await viewer.run()

try:
    await main()
    # asyncio.run(main())
except KeyboardInterrupt:
    print("\nStream stopped.")
except asyncio.CancelledError:
    pass